In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.models import Sequential
from keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, Flatten, TimeDistributed, ConvLSTM2D, Reshape
import tensorflow as tf
import sklearn.metrics as sm
import keras
from keras.losses import huber_loss


from keras import backend as K

Using TensorFlow backend.


In [2]:
data = pd.read_csv("train.csv")

In [3]:
dtest = pd.read_csv("test.csv")

In [4]:
data.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,0,0,0,0


In [5]:
data =  data.drop(["ID","X0","X1","X2","X3","X4","X5","X6","X8" ], axis = 1)

In [6]:
data.head()

,y,X10,X11,X12,X13,X14,X15,X16,X17,X18,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,130.81,0,0,0,1,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
1,88.53,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
2,76.26,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
3,80.62,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,78.02,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
data.shape

(4209, 369)

In [8]:
dtest

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,1,az,v,n,f,d,t,a,w,0,...,0,0,0,1,0,0,0,0,0,0
1,2,t,b,ai,a,d,b,g,y,0,...,0,0,1,0,0,0,0,0,0,0
2,3,az,v,as,f,d,a,j,j,0,...,0,0,0,1,0,0,0,0,0,0
3,4,az,l,n,f,d,z,l,n,0,...,0,0,0,1,0,0,0,0,0,0
4,5,w,s,as,c,d,y,i,m,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4204,8410,aj,h,as,f,d,aa,j,e,0,...,0,0,0,0,0,0,0,0,0,0
4205,8411,t,aa,ai,d,d,aa,j,y,0,...,0,1,0,0,0,0,0,0,0,0
4206,8413,y,v,as,f,d,aa,d,w,0,...,0,0,0,0,0,0,0,0,0,0
4207,8414,ak,v,as,a,d,aa,c,q,0,...,0,0,1,0,0,0,0,0,0,0


In [9]:
dtest =  dtest.drop(["ID","X0","X1","X2","X3","X4","X5","X6","X8" ], axis = 1)

In [10]:
dtest.head() 

,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [11]:
dtest.shape

(4209, 368)

In [12]:
X_train, y_train = data.values[0:3499,1:], data.values[0:3499, :1].ravel()
X_valid, y_valid = data.values[3500:4208,1:], data.values[3500:4208, :1].ravel()
X_test = dtest.values[:, :]

In [13]:
print("X_train----",X_train.shape)
print("y_train----",y_train.shape)      
print("X_valid----",X_valid.shape)
print("y_valid----",y_valid.shape)
print("X_test-----",X_test.shape)

X_train---- (3499, 368)
y_train---- (3499,)
X_valid---- (708, 368)
y_valid---- (708,)
X_test----- (4209, 368)


In [14]:
X_train[0]

array([0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1.,
       0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       1., 0., 0., 0., 0.

In [15]:
y_train[0]

130.81

In [16]:
# y_train = min_max_scaler.fit_transform(y_train)


In [17]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [18]:
def metrics(pred, y_test):
    evs = sm.explained_variance_score(y_test, pred)
    me = sm.max_error(y_test, pred)
    mae = sm.mean_absolute_error(y_test, pred)
    mse = sm.mean_squared_error(y_test, pred)
    rmse = np.sqrt(mse)
    #msle = sm.mean_squared_log_error(y_test, pred)
    m_ae = sm.median_absolute_error(y_test, pred)
    r2 = sm.r2_score(y_test, pred)
    #mpd = sm.mean_poisson_deviance(y_test, pred)
    #mgd = sm.mean_gamma_deviance(y_test, pred)
    mape = mean_absolute_percentage_error(pred, y_test)
    return({'Explained Variance Score': evs,
            'Max Error': me,
            'Mean Absolute Error': mae,
            'Mean Squared Error': mse,
            'Root Mean Squared Error': rmse,
            #'Mean Squared Log Error': msle,
            'Median Absolute Error': m_ae,
            'R² Score': r2,
            #'Mean Poisson Deviance': mpd,
            #'Mean Gamma Deviance': mgd,
            'Mean Absolute Percentage Error': mape
            })

In [19]:
subsequences = 16
timesteps = X_train.shape[1]//subsequences

In [20]:
timesteps 

23

In [21]:
X_train = X_train.reshape((X_train.shape[0], subsequences, 1, timesteps, 1))
X_valid = X_valid.reshape((X_valid.shape[0], subsequences, 1, timesteps, 1))
X_test  = X_test.reshape((X_test.shape[0], subsequences, 1, timesteps, 1))

In [22]:
X_train.shape

(3499, 16, 1, 23, 1)

In [23]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor="loss", patience=50)

In [24]:
modelConvLSTM = Sequential()
modelConvLSTM.add(ConvLSTM2D(filters=64, kernel_size=(1,2), activation='relu', return_sequences=True,input_shape=(X_train.shape[1], 1, X_train.shape[3], X_train.shape[4])))
modelConvLSTM.add(ConvLSTM2D(filters=32, kernel_size=(1,2), activation='relu'))
modelConvLSTM.add(Flatten())
modelConvLSTM.add(Dense(64))
modelConvLSTM.add(Dense(32))
modelConvLSTM.add(Dense(1))
modelConvLSTM.compile(optimizer='adam', loss="huber_loss")
history = modelConvLSTM.fit(X_train, y_train, batch_size=512, epochs=700, verbose=2, callbacks=[early_stop], validation_data = (X_valid, y_valid))

Train on 3499 samples, validate on 708 samples
Epoch 1/700
 - 14s - loss: 100.3774 - val_loss: 98.1014
Epoch 2/700
 - 12s - loss: 98.8591 - val_loss: 93.4738
Epoch 3/700
 - 11s - loss: 80.7156 - val_loss: 12.2144
Epoch 4/700
 - 10s - loss: 37.6997 - val_loss: 44.1166
Epoch 5/700
 - 10s - loss: 35.4292 - val_loss: 15.1573
Epoch 6/700
 - 11s - loss: 18.6540 - val_loss: 18.0773
Epoch 7/700
 - 10s - loss: 14.7664 - val_loss: 11.2950
Epoch 8/700
 - 10s - loss: 11.8276 - val_loss: 9.5821
Epoch 9/700
 - 10s - loss: 10.6802 - val_loss: 10.0042
Epoch 10/700
 - 11s - loss: 10.4086 - val_loss: 10.2098
Epoch 11/700
 - 11s - loss: 10.2599 - val_loss: 9.0468
Epoch 12/700
 - 11s - loss: 9.9957 - val_loss: 9.1253
Epoch 13/700
 - 11s - loss: 9.8780 - val_loss: 9.4162
Epoch 14/700
 - 11s - loss: 9.8987 - val_loss: 9.0202
Epoch 15/700
 - 11s - loss: 9.7635 - val_loss: 8.9910
Epoch 16/700
 - 11s - loss: 9.7652 - val_loss: 9.1599
Epoch 17/700
 - 11s - loss: 9.7927 - val_loss: 9.0811
Epoch 18/700
 - 26s - l

Epoch 151/700
 - 45s - loss: 5.6722 - val_loss: 4.4477
Epoch 152/700
 - 46s - loss: 5.4295 - val_loss: 4.4415
Epoch 153/700
 - 45s - loss: 5.4421 - val_loss: 4.5011
Epoch 154/700
 - 45s - loss: 5.3816 - val_loss: 4.6118
Epoch 155/700
 - 45s - loss: 5.2985 - val_loss: 4.4993
Epoch 156/700
 - 45s - loss: 5.2638 - val_loss: 5.3700
Epoch 157/700
 - 46s - loss: 5.4166 - val_loss: 4.6010
Epoch 158/700
 - 46s - loss: 5.2182 - val_loss: 4.7107
Epoch 159/700
 - 45s - loss: 5.3326 - val_loss: 5.2591
Epoch 160/700
 - 45s - loss: 5.4421 - val_loss: 5.3925
Epoch 161/700
 - 45s - loss: 5.4771 - val_loss: 5.3826
Epoch 162/700
 - 45s - loss: 5.4656 - val_loss: 4.7664
Epoch 163/700
 - 45s - loss: 5.3219 - val_loss: 5.7256
Epoch 164/700
 - 45s - loss: 5.6204 - val_loss: 4.5059
Epoch 165/700
 - 46s - loss: 5.4676 - val_loss: 4.4242
Epoch 166/700
 - 45s - loss: 5.3718 - val_loss: 4.6578
Epoch 167/700
 - 45s - loss: 5.3471 - val_loss: 4.9749
Epoch 168/700
 - 45s - loss: 5.2184 - val_loss: 5.0354
Epoch 169/

Epoch 300/700
 - 45s - loss: 4.9219 - val_loss: 4.0764
Epoch 301/700
 - 45s - loss: 4.9728 - val_loss: 4.3310
Epoch 302/700
 - 46s - loss: 4.9459 - val_loss: 5.2066
Epoch 303/700
 - 46s - loss: 5.1081 - val_loss: 4.9282
Epoch 304/700
 - 45s - loss: 5.0428 - val_loss: 4.1408
Epoch 305/700
 - 45s - loss: 4.8924 - val_loss: 4.2655
Epoch 306/700
 - 45s - loss: 4.8658 - val_loss: 4.1599
Epoch 307/700
 - 45s - loss: 4.8293 - val_loss: 4.8039
Epoch 308/700
 - 45s - loss: 4.8782 - val_loss: 4.2754
Epoch 309/700
 - 45s - loss: 4.8291 - val_loss: 4.0512
Epoch 310/700
 - 45s - loss: 4.8543 - val_loss: 4.3061
Epoch 311/700
 - 45s - loss: 4.7711 - val_loss: 4.2188
Epoch 312/700
 - 45s - loss: 4.7388 - val_loss: 4.1815
Epoch 313/700
 - 45s - loss: 4.7571 - val_loss: 4.0892
Epoch 314/700
 - 45s - loss: 4.8219 - val_loss: 4.2537
Epoch 315/700
 - 46s - loss: 4.8459 - val_loss: 4.8430
Epoch 316/700
 - 45s - loss: 4.9273 - val_loss: 4.3805
Epoch 317/700
 - 45s - loss: 4.7487 - val_loss: 4.1306
Epoch 318/

Epoch 449/700
 - 45s - loss: 4.8246 - val_loss: 4.1251
Epoch 450/700
 - 46s - loss: 4.6553 - val_loss: 4.0355
Epoch 451/700
 - 45s - loss: 4.6276 - val_loss: 4.4214
Epoch 452/700
 - 46s - loss: 4.6293 - val_loss: 3.9722
Epoch 453/700
 - 45s - loss: 4.6636 - val_loss: 4.4240
Epoch 454/700
 - 45s - loss: 4.6717 - val_loss: 4.4103
Epoch 455/700
 - 45s - loss: 4.7449 - val_loss: 3.9398
Epoch 456/700
 - 45s - loss: 4.7535 - val_loss: 4.0815
Epoch 457/700
 - 45s - loss: 4.6648 - val_loss: 4.5368
Epoch 458/700
 - 45s - loss: 4.6059 - val_loss: 4.1365
Epoch 459/700
 - 46s - loss: 4.6041 - val_loss: 4.1656
Epoch 460/700
 - 46s - loss: 4.5592 - val_loss: 4.0563
Epoch 461/700
 - 47s - loss: 4.5725 - val_loss: 4.1622
Epoch 462/700
 - 47s - loss: 4.5579 - val_loss: 4.0393
Epoch 463/700
 - 46s - loss: 4.5473 - val_loss: 4.3847
Epoch 464/700
 - 47s - loss: 4.5831 - val_loss: 4.1949
Epoch 465/700
 - 47s - loss: 4.5644 - val_loss: 4.2227
Epoch 466/700
 - 47s - loss: 4.5700 - val_loss: 4.0295
Epoch 467/

Epoch 598/700
 - 34s - loss: 4.4695 - val_loss: 4.1713
Epoch 599/700
 - 34s - loss: 4.5044 - val_loss: 4.3897
Epoch 600/700
 - 34s - loss: 4.4607 - val_loss: 4.4797
Epoch 601/700
 - 34s - loss: 4.5170 - val_loss: 4.3627
Epoch 602/700
 - 34s - loss: 4.5220 - val_loss: 3.9837
Epoch 603/700
 - 34s - loss: 4.4608 - val_loss: 4.2783
Epoch 604/700
 - 34s - loss: 4.4581 - val_loss: 4.2441
Epoch 605/700
 - 34s - loss: 4.4638 - val_loss: 4.1427
Epoch 606/700
 - 34s - loss: 4.4967 - val_loss: 4.4404
Epoch 607/700
 - 34s - loss: 4.4506 - val_loss: 4.1767
Epoch 608/700
 - 34s - loss: 4.4710 - val_loss: 4.1419
Epoch 609/700
 - 34s - loss: 4.4783 - val_loss: 4.0741
Epoch 610/700
 - 34s - loss: 4.5045 - val_loss: 4.1849
Epoch 611/700
 - 34s - loss: 4.4339 - val_loss: 4.0438
Epoch 612/700
 - 34s - loss: 4.4685 - val_loss: 3.9465
Epoch 613/700
 - 34s - loss: 4.5849 - val_loss: 4.1559
Epoch 614/700
 - 34s - loss: 4.5505 - val_loss: 5.4626
Epoch 615/700
 - 34s - loss: 4.7786 - val_loss: 4.0283
Epoch 616/

In [25]:
ConvLSTMpred = modelConvLSTM.predict(X_valid, verbose=0)
ConvLSTMpred = ConvLSTMpred.reshape((ConvLSTMpred.shape[0]))
ConvLSTMresults = metrics(ConvLSTMpred, y_valid)
ConvLSTMresults

{'Explained Variance Score': 0.667441093049198,
 'Max Error': 41.20085632324219,
 'Mean Absolute Error': 5.049645117204742,
 'Mean Squared Error': 45.42418250680345,
 'Root Mean Squared Error': 6.7397464719975515,
 'Median Absolute Error': 4.281689758300786,
 'R² Score': 0.6551291460523878,
 'Mean Absolute Percentage Error': 5.035695447646363}

In [26]:
ConvLSTMpred = modelConvLSTM.predict(X_test, verbose=0)
ConvLSTMpred = ConvLSTMpred.reshape((ConvLSTMpred.shape[0]))


In [27]:

ConvLSTMpred 


array([ 77.34828 ,  91.77606 ,  75.27989 , ...,  92.2388  , 112.730385,
        92.28337 ], dtype=float32)